#Format Data for Experiment in ABM environment

In [ ]:
#Import Packages
#Needed for moving to output
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as pl

#Analysis tumor-tcell modules needed
from tumor_tcell.library.population_plots import death_group_plot
from tumor_tcell.library.population_plots import population_group_plot

In [ ]:
#Get csv saved in experiment id library
analysis_dir = '/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/'
save_dir = analysis_dir+'Multiple_analysis/'

In [ ]:
#80 replicates
analysis_out_dir = save_dir + 'PD1_80_in_rep' #####################
os.makedirs(analysis_out_dir, exist_ok=True)

experiment_list = [ 'tumor_tcell_20210608.202800/',  
                   'tumor_tcell_20210608.202806/', ] #####################
experiment_rename = {  
    'tumor_tcell_20210608.202800/':'80% PD1+ inside 1',
    'tumor_tcell_20210608.202806/':'80% PD1+ inside 2',
                    } ######################

In [ ]:
#Create a list of dataframes for concatenation
df_tumor_death_list = []
df_tcell_death_list = []
tumor_plot_list = []
tcell_plot_list = []

#read in dataframes
for experiment in experiment_list:
   
    experiment_id = experiment
    experiment_dir = analysis_dir+experiment_id
    os.chdir(experiment_dir)

    df_tumor_death = pd.read_csv('tumor_death.csv')
    df_tumor_death['experiment_name'] = df_tumor_death['experiment_id'].map(experiment_rename)
    df_tumor_death_list.append(df_tumor_death)
    
    tumor_plot = pd.read_csv('tumor_plot.csv')
    tumor_plot['experiment_name'] = tumor_plot['experiment_id'].map(experiment_rename)
    tumor_plot_list.append(tumor_plot)
    
    df_tcell_death = pd.read_csv('tcell_death.csv')
    df_tcell_death['experiment_name'] = df_tcell_death['experiment_id'].map(experiment_rename)
    df_tcell_death_list.append(df_tcell_death)
    
    tcell_plot = pd.read_csv('tcell_plot.csv')
    tcell_plot['experiment_name'] = tcell_plot['experiment_id'].map(experiment_rename)
    tcell_plot_list.append(tcell_plot)
    
tcell_plot_list[0];

## Grouped Analysis Graphs

In [ ]:
death_group_plot(death_plot_list = df_tcell_death_list, out_dir=analysis_out_dir, save_name='Tcells')

In [ ]:
death_group_plot(death_plot_list = df_tumor_death_list, out_dir=analysis_out_dir, save_name='Tumors')

In [ ]:
population_group_plot(cell_plot_list = tcell_plot_list, cell_states = ['PD1n','PD1p'], out_dir=analysis_out_dir, save_name='Tcells')

In [ ]:
population_group_plot(cell_plot_list = tumor_plot_list, cell_states = ['PDL1n','PDL1p'], out_dir=analysis_out_dir, save_name='Tumor')

# Saved Analyses

In [ ]:
#80% PD1p inside replicates
analysis_out_dir = save_dir + 'PD1_80_in_replicates' #####################
os.makedirs(analysis_out_dir, exist_ok=True)

experiment_list = [ 'tumor_tcell_20210501.171624/', 'tumor_tcell_20210514.165353/',
                   'tumor_tcell_20210514.165410/', 'tumor_tcell_20210517.182737/',
                   'tumor_tcell_20210517.184351/'] #####################
experiment_rename = { 
                     'tumor_tcell_20210501.171624/':'80% PD1+ x1', 
                     'tumor_tcell_20210514.165353/':'80% PD1+ x2',
                     'tumor_tcell_20210514.165410/':'80% PD1+ x3',
                     'tumor_tcell_20210517.182737/':'80% PD1+ x4',
                     'tumor_tcell_20210517.184351/':'80% PD1+ x5', 
                    } ######################

In [ ]:
#20% PD1p inside replicates
analysis_out_dir = save_dir + 'PD1_20_in_replicates' #####################
os.makedirs(analysis_out_dir, exist_ok=True)

experiment_list = [ 'tumor_tcell_20210430.175059/', 'tumor_tcell_20210514.165656/',
                   'tumor_tcell_20210514.165554/', 'tumor_tcell_20210517.182724/',
                   'tumor_tcell_20210517.182642/'] #####################
experiment_rename = { 
    'tumor_tcell_20210430.175059/':'20% PD1+ x1', 
    'tumor_tcell_20210514.165656/':'20% PD1+ x2',
    'tumor_tcell_20210514.165554/':'20% PD1+ x3', 
    'tumor_tcell_20210517.182724/':'20% PD1+ x4',
    'tumor_tcell_20210517.182642/':'20% PD1+ x5',
                    } ######################

In [ ]:
#80 vs. 20 vs. control inside
analysis_out_dir = save_dir + 'PD1_2080_in' #####################
os.makedirs(analysis_out_dir, exist_ok=True)

experiment_list = [ 'tumor_tcell_20210501.171624', 'tumor_tcell_20210430.175059/', 
                   'tumor_tcell_20210511.172757/'] #####################
experiment_rename = { 
    'tumor_tcell_20210501.171624/':'80% PD1+', 
    'tumor_tcell_20210430.175059/':'20% PD1+',
    'tumor_tcell_20210511.172757/':'No T cells',
                    } ######################

In [ ]:
#80 vs. 20 vs. control outside
analysis_out_dir = save_dir + 'PD1_2080_out' #####################
os.makedirs(analysis_out_dir, exist_ok=True)

experiment_list = [ 'tumor_tcell_20210503.163607', 'tumor_tcell_20210429.132351/', 
                   'tumor_tcell_20210511.172757/'] #####################
experiment_rename = { 
    'tumor_tcell_20210429.132351/':'20% PD1+', 
    'tumor_tcell_20210503.163607/':'80% PD1+',
    'tumor_tcell_20210511.172757/':'No T cells',
                    } ######################

In [ ]:
#20 outside vs. inside
analysis_out_dir = save_dir + 'PD1_20_inout' #####################
os.makedirs(analysis_out_dir, exist_ok=True)

experiment_list = [ 'tumor_tcell_20210429.132351/', 
                   'tumor_tcell_20210430.175059/', ] #####################
experiment_rename = { 
    'tumor_tcell_20210429.132351/':'20% PD1+ outside', 
    'tumor_tcell_20210430.175059/':'20% PD1+ inside', 
                    } ######################

In [ ]:
#80 outside vs. inside
analysis_out_dir = save_dir + 'PD1_80_inout' #####################
os.makedirs(analysis_out_dir, exist_ok=True)

experiment_list = [ 'tumor_tcell_20210503.163607/',  
                   'tumor_tcell_20210501.171624/', ] #####################
experiment_rename = {  
    'tumor_tcell_20210503.163607/':'80% PD1+ outside',
    'tumor_tcell_20210501.171624/':'80% PD1+ inside',
                    } ######################

In [ ]:
#80 vs. 20 vs. control outside vs. inside
analysis_out_dir = save_dir + 'PD1_2080_inout' #####################
os.makedirs(analysis_out_dir, exist_ok=True)

experiment_list = [ 'tumor_tcell_20210503.163607', 'tumor_tcell_20210429.132351/', 
                  'tumor_tcell_20210501.171624', 
                   'tumor_tcell_20210430.175059/', 'tumor_tcell_20210511.172757/',] #####################
experiment_rename = { 
    'tumor_tcell_20210429.132351/':'20% PD1+ outside', 
    'tumor_tcell_20210503.163607/':'80% PD1+ outside',
    'tumor_tcell_20210511.172757/':'No T cells',
    'tumor_tcell_20210501.171624/':'80% PD1+ inside', 
    'tumor_tcell_20210430.175059/':'20% PD1+ inside',
                    } ######################